In [1]:
!pip install google-genai

In [ ]:
# Célula 1: Instalação das Bibliotecas
!pip install google-generativeai google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install ipywidgets # Para uma interface mais interativa (opcional)

# Célula 2: Autenticação (Google AI Studio e APIs Google Workspace)
import os
from google.colab import userdata
import google.generativeai as genai
from googleapiclient.discovery import build
from google.oauth2 import credentials

# Autenticação para Gemini (Google AI Studio)
# Use userdata para obter a chave da API com segurança
try:
    # Tenta obter a chave da API do userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY  # Define como variável de ambiente (opcional, mas útil)
    print("Chave da API obtida com sucesso do userdata!")

    # Configura o Gemini com a chave da API
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro')  # Ou 'gemini-pro-vision' se precisar de imagens
    print("Gemini configurado com sucesso!")

except KeyError:
    print("Erro: Chave da API 'GOOGLE_API_KEY' não encontrada no userdata.")
    print("Por favor, adicione a chave da API em 'Segredos' no Google Colab (menu à esquerda).")
    GOOGLE_API_KEY = None  # Define como None para evitar erros posteriores
    model = None # Define como None para evitar erros posteriores

except Exception as e:
    print(f"Erro ao obter ou configurar a chave da API: {e}")
    GOOGLE_API_KEY = None # Define como None para evitar erros posteriores
    model = None # Define como None para evitar erros posteriores


# Autenticação para APIs do Google Workspace (Drive, Calendar, etc.)
# Este código usa o fluxo de autenticação do Colab. Ele exibirá um link para você fazer login.
SCOPES = ['https://www.googleapis.com/auth/drive.readonly', # Exemplo: Acesso de leitura ao Drive
          'https://www.googleapis.com/auth/calendar.readonly', # Exemplo: Acesso de leitura ao Calendar
          'https://www.googleapis.com/auth/gmail.readonly'] # Exemplo: Acesso de leitura ao Gmail

creds, _ = google.auth.default(scopes=SCOPES)

# Construir serviços da API
try:
    drive_service = build('drive', 'v3', credentials=creds)
    calendar_service = build('calendar', 'v3', credentials=creds)
    gmail_service = build('gmail', 'v1', credentials=creds) # Cuidado com a API do Gmail - veja as considerações abaixo
    print("Autenticação bem-sucedida nas APIs do Google Workspace!")
except Exception as e:
    print(f"Erro na autenticação das APIs do Google Workspace: {e}")

# Célula 3: Agente InsightCliente (AIC)

def rodar_agente_insight_cliente(texto_briefing):
    """
    Extrai informações de um briefing usando Gemini.

    Args:
        texto_briefing: O texto do briefing do cliente.

    Returns:
        Um dicionário com o briefing estruturado, perguntas de esclarecimento e resumo de pesquisa de mercado.
    """
    prompt = f"""
    Analise o seguinte briefing do cliente:
    {texto_briefing}

    1.  Resuma os pontos principais e identifique os requisitos explícitos e implícitos.
    2.  Liste quaisquer ambiguidades ou informações faltantes.
    3.  Gere 3 perguntas de acompanhamento para esclarecer os pontos.
    4.  Crie um resumo de pesquisa de mercado (máximo 5 frases) para enriquecer o briefing.

    Formate a resposta em um dicionário Python com as chaves: "resumo", "requisitos", "ambiguidades", "perguntas", "resumo_pesquisa".
    """

    try:
      if model is None:
        raise ValueError("O modelo Gemini não foi inicializado corretamente. Verifique sua chave de API.")
      response = model.generate_content(prompt)
      # Aqui, idealmente, você usaria eval() para transformar a string em um dicionário Python.
      # **AVISO:** eval() pode ser perigoso se você não controlar a entrada.
      # Em vez de eval(), você pode usar expressões regulares ou json.loads() se a formatação for consistente.
      # Para o protótipo, vamos imprimir a resposta e deixar você copiar/colar e formatar manualmente.
      print("Resposta do Gemini (AIC):")
      print(response.text)
      # Implementar a lógica para analisar a string de resposta e criar o dicionário
      # ...
      return response.text # Retorna a resposta bruta (para demonstração)
    except Exception as e:
        print(f"Erro no AIC: {e}")
        return None

# Célula 4: Agente PlanejadorEstrategico (APE)

def rodar_agente_planejador_estrategico(briefing_estruturado, personas_equipe, prazo_macro):
    """
    Cria um plano de projeto inicial usando Gemini.

    Args:
        briefing_estruturado: O briefing estruturado (saída do AIC).
        personas_equipe: Uma lista das personas da equipe (ex: "Redator", "DA").
        prazo_macro: O prazo geral do projeto.

    Returns:
        Um dicionário com o esqueleto do plano de projeto, sugestão de cronograma e rascunho de atribuições.
    """

    prompt = f"""
    Com base no seguinte briefing estruturado:
    {briefing_estruturado}

    E considerando as seguintes personas na equipe:
    {personas_equipe}

    E o prazo macro de:
    {prazo_macro}

    1.  Sugira uma estrutura de fases e tarefas principais (ex: "Fase de Layout: Wireframe Home", "Conteúdo LP1").
    2.  Rascunhe atribuições iniciais (ex: "Redator: Conteúdo Home", "DA: Moodboard").
    3.  Crie um esqueleto de cronograma com marcos principais.

    Formate a resposta em um dicionário Python com as chaves: "fases", "atribuicoes", "cronograma".
    """

    try:
      if model is None:
        raise ValueError("O modelo Gemini não foi inicializado corretamente. Verifique sua chave de API.")
      response = model.generate_content(prompt)
      print("Resposta do Gemini (APE):")
      print(response.text)
      # Implementar a lógica para analisar a string de resposta e criar o dicionário
      # ...
      return response.text # Retorna a resposta bruta (para demonstração)
    except Exception as e:
        print(f"Erro no APE: {e}")
        return None

# Célula 5: Agente FacilitadorCriativo (AFC)

def rodar_agente_facilitador_criativo(briefing, tipo_tarefa, descricao_tarefa):
    """
    Auxilia na criação de conteúdo ou design usando Gemini.

    Args:
        briefing: O briefing do cliente.
        tipo_tarefa: "redacao" ou "design".
        descricao_tarefa: Uma descrição específica da tarefa (ex: "gerar headlines para a home", "buscar referências de design minimalista").

    Returns:
        Um dicionário com ideias de conteúdo ou referências visuais.
    """

    prompt = f"""
    Com base no seguinte briefing:
    {briefing}

    E para a seguinte tarefa de {tipo_tarefa}: {descricao_tarefa}

    Gere ideias criativas e relevantes. Se for redação, sugira headlines e CTAs. Se for design, busque referências visuais e paletas de cores.
    """

    try:
      if model is None:
        raise ValueError("O modelo Gemini não foi inicializado corretamente. Verifique sua chave de API.")
      response = model.generate_content(prompt)
      print("Resposta do Gemini (AFC):")
      print(response.text)
      # Implementar a lógica para analisar a string de resposta e criar o dicionário
      # ...
      return response.text # Retorna a resposta bruta (para demonstração)
    except Exception as e:
        print(f"Erro no AFC: {e}")
        return None

# Célula 6: Agente GuardiaoDoEscopo (AGE)

def rodar_agente_guardiao_do_escopo(escopo_original, solicitacao_alteracao):
    """
    Analisa o impacto de uma solicitação de alteração no escopo do projeto.

    Args:
        escopo_original: O escopo original do projeto (saída do AIC/APE).
        solicitacao_alteracao: A descrição da solicitação de alteração.

    Returns:
        Um dicionário com a análise de impacto e sugestões de opções.
    """

    prompt = f"""
    O escopo original do projeto é:
    {escopo_original}

    O cliente solicitou a seguinte alteração:
    {solicitacao_alteracao}

    1.  Compare a solicitação com o escopo original.
    2.  Analise o impacto potencial no prazo e no custo.
    3.  Sugira opções para lidar com a alteração (ex: manter escopo e prazo, ou aceitar a alteração com novo prazo/custo).
    4.  Crie um rascunho de comunicação para o cliente explicando o impacto e as opções.

    Formate a resposta em um dicionário Python com as chaves: "analise_impacto", "opcoes", "rascunho_comunicacao".
    """

    try:
      if model is None:
        raise ValueError("O modelo Gemini não foi inicializado corretamente. Verifique sua chave de API.")
      response = model.generate_content(prompt)
      print("Resposta do Gemini (AGE):")
      print(response.text)
      # Implementar a lógica para analisar a string de resposta e criar o dicionário
      # ...
      return response.text # Retorna a resposta bruta (para demonstração)
    except Exception as e:
        print(f"Erro no AGE: {e}")
        return None

# Célula 7: Agente ComunicadorProativo (ACP)

def rodar_agente_comunicador_proativo(tipo_comunicacao, detalhes):
    """
    Gera rascunhos de comunicações (e-mails, pautas, resumos) usando Gemini.

    Args:
        tipo_comunicacao: "email_status", "pauta_reuniao", "resumo_comunicacao".
        detalhes: Detalhes específicos para cada tipo de comunicação.

    Returns:
        Um rascunho da comunicação.
    """

    prompt = f"""
    Gere um rascunho para o seguinte tipo de comunicação: {tipo_comunicacao}

    Com os seguintes detalhes: {detalhes}

    Se for um e-mail de status, inclua o progresso das tarefas, os próximos passos e quaisquer bloqueios.
    Se for uma pauta de reunião, inclua os tópicos a serem discutidos, os participantes e o objetivo da reunião.
    Se for um resumo de comunicação, resuma os pontos principais da discussão.
    """

    try:
      if model is None:
        raise ValueError("O modelo Gemini não foi inicializado corretamente. Verifique sua chave de API.")
      response = model.generate_content(prompt)
      print("Resposta do Gemini (ACP):")
      print(response.text)
      return response.text
    except Exception as e:
        print(f"Erro no ACP: {e}")
        return None

# Célula 8: Agente VerificadorTecnico (AVT)

def rodar_agente_verificador_tecnico(duvida_tecnica):
    """
    Responde a dúvidas técnicas usando Gemini e pesquisa na web.

    Args:
        duvida_tecnica: A dúvida técnica sobre Webflow, HubSpot, etc.

    Returns:
        Uma resposta à dúvida técnica e links para documentação relevante.
    """

    prompt = f"""
    Responda à seguinte dúvida técnica:
    {duvida_tecnica}

    Inclua exemplos de código, links para documentação oficial e melhores práticas.
    """

    try:
      if model is None:
        raise ValueError("O modelo Gemini não foi inicializado corretamente. Verifique sua chave de API.")
      response = model.generate_content(prompt)
      print("Resposta do Gemini (AVT):")
      print(response.text)
      # Aqui você pode adicionar lógica para pesquisar na web usando a Google Search API
      # e adicionar os links relevantes à resposta.
      return response.text
    except Exception as e:
        print(f"Erro no AVT: {e}")
        return None

# Célula 9: Fluxo de Trabalho de Exemplo (Simulação do GP)

print("Simulando o fluxo de trabalho do Gerente de Projeto:")

# 1. Início do Projeto:

print("\n--- Início do Projeto ---")
briefing_cliente = input("Insira o briefing inicial do cliente: ")
saida_aic = rodar_agente_insight_cliente(briefing_cliente)

if saida_aic:
    print("\nSaída do Agente InsightCliente (AIC):")
    print(saida_aic)

    # Aqui você precisaria analisar a saída do AIC para extrair o briefing estruturado
    # Para simplificar, vamos pedir ao usuário para inserir manualmente.
    briefing_estruturado = input("Insira o briefing estruturado (extraído da saída do AIC): ")
    personas_equipe = ["Redator", "Diretor de Arte", "Programador"]
    prazo_macro = "4 semanas"
    saida_ape = rodar_agente_planejador_estrategico(briefing_estruturado, personas_equipe, prazo_macro)

    if saida_ape:
        print("\nSaída do Agente PlanejadorEstrategico (APE):")
        print(saida_ape)

        # Novamente, você precisaria analisar a saída do APE para extrair o plano de projeto
        # Para simplificar, vamos simular a criação de um e-mail de kick-off
        detalhes_email_kickoff = f"Plano de projeto: {saida_ape}. Próxima reunião: [data]."
        rascunho_email = rodar_agente_comunicador_proativo("email_status", detalhes_email_kickoff)

        if rascunho_email:
            print("\nRascunho do e-mail de kick-off (Agente ComunicadorProativo):")
            print(rascunho_email)

# 2. Durante a Execução (Ex: Fase de Layout):

print("\n--- Durante a Execução (Fase de Layout) ---")
duvida_webflow = input("O Programador Full Stack tem uma dúvida sobre Webflow: ")
resposta_avt = rodar_agente_verificador_tecnico(duvida_webflow)

if resposta_avt:
    print("\nResposta do Agente VerificadorTecnico (AVT):")
    print(resposta_avt)

# 3. Solicitação de Alteração:

print("\n--- Solicitação de Alteração ---")
escopo_original = briefing_estruturado # Usando o briefing estruturado como escopo original (simplificado)
solicitacao_alteracao = input("O cliente solicitou uma alteração no layout: ")
analise_escopo = rodar_agente_guardiao_do_escopo(escopo_original, solicitacao_alteracao)

if analise_escopo:
    print("\nAnálise de Escopo (Agente GuardiaoDoEscopo):")
    print(analise_escopo)

    # Aqui você usaria a análise de escopo para comunicar o impacto ao cliente
    # e, se aprovado, ajustar o cronograma (usando o APE e ACP).

print("\n--- Fim da Simulação ---")

Chave da API obtida com sucesso do userdata!
Gemini configurado com sucesso!
Autenticação bem-sucedida nas APIs do Google Workspace!
Simulando o fluxo de trabalho do Gerente de Projeto:

--- Início do Projeto ---
